In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.models import resnet34
import torch.nn as nn
import torch.optim as optim
import time

In [6]:
# Parameters
data_dir = '/content/drive/MyDrive/JPEGImages'
batch_size = 32
num_workers = 4
learning_rate = 0.001
num_epochs = 10
print_step = 20  # Print loss every 20 steps

# 1. Data Loading
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [7]:
# 2. Model Setup
model = resnet34(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 50)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
torch.backends.cudnn.benchmark = True  # Enable cuDNN auto-tuner

In [8]:
# 3. Model Training
for epoch in range(num_epochs):
    start_time = time.time()  # Start timing for the epoch
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Print every 20 steps
        if (i + 1) % print_step == 0:
            print(f'Epoch {epoch+1}, Step {i+1}, Loss: {loss.item()}')

    end_time = time.time()  # End timing for the epoch
    epoch_duration = end_time - start_time
    print(f'Epoch {epoch+1} completed in {epoch_duration:.2f} seconds, Average Loss: {running_loss / len(train_loader)}')



Epoch 1, Step 20, Loss: 2.2346882820129395
Epoch 1, Step 40, Loss: 2.900305986404419
Epoch 1, Step 60, Loss: 3.0036957263946533
Epoch 1, Step 80, Loss: 2.186126947402954
Epoch 1, Step 100, Loss: 2.0259175300598145
Epoch 1, Step 120, Loss: 1.6902984380722046
Epoch 1, Step 140, Loss: 1.867910623550415
Epoch 1, Step 160, Loss: 1.8519620895385742
Epoch 1, Step 180, Loss: 1.5505088567733765
Epoch 1, Step 200, Loss: 1.2665947675704956
Epoch 1, Step 220, Loss: 1.7678723335266113
Epoch 1, Step 240, Loss: 2.639819860458374
Epoch 1 completed in 1473.16 seconds, Average Loss: 2.166012548198996
Epoch 2, Step 20, Loss: 1.5490702390670776
Epoch 2, Step 40, Loss: 1.4684022665023804
Epoch 2, Step 60, Loss: 1.3083747625350952
Epoch 2, Step 80, Loss: 0.9680299162864685
Epoch 2, Step 100, Loss: 1.9287347793579102
Epoch 2, Step 120, Loss: 1.027632474899292
Epoch 2, Step 140, Loss: 1.0551236867904663
Epoch 2, Step 160, Loss: 0.746101975440979
Epoch 2, Step 180, Loss: 1.2534312009811401
Epoch 2, Step 200, L

In [11]:
# 4. Model Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total}%')

Accuracy on test set: 65.56580864497329%
